In [0]:
run Lattice_functions.ipynb

# Model 3: Constant Stiffness Model : Force variation

In [0]:
#Generating the lattice
nx = 20 + 1 # 1 added to make the total length = 20*0.01
ny = 20 + 1
lx = 0.05
ly = 0.05
k_inPlane = 1e3
nz = 3
lz = 0.01
k_vertical = 1.4e5

M3 = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
M3 = add_braces_to_rectangular_lattice(M3, k_inPlane, '|')
M3 = add_layers_in_z_direction(M3, nz, lz, k_vertical, k_vertical, [1, 2, 3, 4])

#setting z of the middle layer to zero and deriving support conditions:
min_x = M3.nodes[1].x
min_y = M3.nodes[1].y
max_x = M3.nodes[1].x
max_y = M3.nodes[1].y

for i in M3.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

#Adding support
for i in M3.nodes:
    if(i.z == 0):
        if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
            i.s_x = 1
            i.s_y = 1
            i.s_z = 1
            M3.support_x.append(i.n)
            M3.support_y.append(i.n)
            M3.support_z.append(i.n)


mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny

In [0]:
#Optimization preparation:
x0 = []
y0 = []
z0 = []
for i in M3.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
coord0 = x0 + y0 + z0
nNodes = int(len(coord0)/3)

#bounds:
lb = -np.inf*np.ones(len(coord0))
ub = np.inf*np.ones(len(coord0))

for i in M3.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

for i in M3.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
for i in M3.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


bounds = Bounds(lb, ub)

#Defining half step force
f0 = Point_load_at_centre(0.5, M3)


#Variable for saving the data
data_M3 = [] #has the coordinates of the energy minimized lattices
forces_M3 = [] #forces applied at the centre
energies_M3 = [] #energies of the optimized lattices
z_centre_M3 = [] #deflection at centre
results_M3 = [] #Outputs from scipy.optimize

In [0]:
for i in range(1, 22):
  print(i)
  f = (i - 1)*f0
  result = minimize(Energy, coord0, args = (f, M3), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
  res = result.x
  res.tolist()
  data_M3.append(res)
  forces_M3.append(max(f))
  energies_M3.append(result.fun)
  z_centre_M3.append(res[2*nNodes + mid_node - 1])
  results_M3.append(result)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [0]:
from numpy import save
save('data_M3.npy', data_M3)
save('forces_M3.npy', forces_M3)
save('energies_M3.npy', energies_M3)
save('z_centre_M3.npy', z_centre_M3)
save('results_M3.npy', results_M3)
save('M3.npy', M3)

# Model 3: Constant AE Model : Force variation

In [0]:
#Generating the lattice
nx = 20 + 1 # 1 added to make the total length = 20*0.01
ny = 20 + 1
lx = 0.05
ly = 0.05
k_inPlane = 10
nz = 3
lz = 0.01
k_vertical = 10

M3_AE = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
M3_AE = add_braces_to_rectangular_lattice(M3_AE, k_inPlane, '|')
M3_AE = add_layers_in_z_direction(M3_AE, nz, lz, k_vertical, k_vertical, [1, 2, 3, 4])

# modifying stiffness wrt AE = constant
AE = 0.9e2
for i in M3_AE.springs:
  i.k = AE/(i.l0)


for i in M3_AE.springs:
  if (abs(M3_AE.nodes[i.i - 1].z - M3_AE.nodes[i.j - 1].z) != 0):
    i.k = i.k*1e2

#setting z of the middle layer to zero and deriving support conditions:
min_x = M3_AE.nodes[1].x
min_y = M3_AE.nodes[1].y
max_x = M3_AE.nodes[1].x
max_y = M3_AE.nodes[1].y

for i in M3_AE.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

#Adding support
for i in M3_AE.nodes:
    if(i.z == 0):
        if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
            i.s_x = 1
            i.s_y = 1
            i.s_z = 1
            M3_AE.support_x.append(i.n)
            M3_AE.support_y.append(i.n)
            M3_AE.support_z.append(i.n)


mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny

In [0]:
#Optimization preparation:
x0 = []
y0 = []
z0 = []
for i in M3_AE.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
coord0 = x0 + y0 + z0
nNodes = int(len(coord0)/3)

#bounds:
lb = -np.inf*np.ones(len(coord0))
ub = np.inf*np.ones(len(coord0))

for i in M3_AE.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

for i in M3_AE.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
for i in M3_AE.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


bounds = Bounds(lb, ub)

#Defining half unit forces
f0 = Point_load_at_centre(0.5, M3_AE)

#Variable for saving the data
data_M3_AE = [] #has the coordinates of the energy minimized lattices
forces_M3_AE = [] #forces applied at the centre
energies_M3_AE = [] #energies of the optimized lattices
z_centre_M3_AE = [] #deflection at centre
results_M3_AE = [] #Outputs from scipy.optimize

In [0]:
for i in range(1, 22):
    print(i)
    f = (i - 1)*f0
    result = minimize(Energy, coord0, args = (f, M3_AE), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
    res = result.x
    res.tolist()
    data_M3_AE.append(res)
    forces_M3_AE.append(max(f))
    energies_M3_AE.append(result.fun)
    z_centre_M3_AE.append(res[2*nNodes + mid_node - 1])
    results_M3_AE.append(result)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [0]:
from numpy import save
save('data_M3_AE.npy', data_M3_AE)
save('forces_M3_AE.npy', forces_M3_AE)
save('energies_M3_AE.npy', energies_M3_AE)
save('z_centre_M3_AE.npy', z_centre_M3_AE)
save('results_M3_AE.npy', results_M3_AE)
save('M3_AE.npy', M3_AE)

# Model 3: Constant Stiffness Model : Thickness variation

In [0]:
#Variable for saving the data
data_M3_t = [] #has the coordinates of the energy minimized lattices
thickness_M3_t = [] #Thickness of the model
energies_M3_t = [] #energies of the optimized lattices
z_centre_M3_t = [] #deflection at centre
results_M3_t = [] #Outputs from scipy.optimize
lat_M3_t = [] #Lattices with varying thickness

In [0]:
#Model 3, thickness varying condition
for i in range(1, 11):
  print(i)
  thickness = 0.005*i

  #Generating the lattice
  nx = 20 + 1 # 1 added to make the total length = 20*0.01
  ny = 20 + 1
  lx = 0.05
  ly = 0.05
  k_inPlane = 1e3
  nz = 3
  lz = thickness
  k_vertical = 1.4e5

  M3_t = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
  M3_t = add_braces_to_rectangular_lattice(M3_t, k_inPlane, '|')
  M3_t = add_layers_in_z_direction(M3_t, nz, lz, k_vertical, k_vertical, [1, 2, 3, 4])

  #setting z of the middle layer to zero:
  min_x = M3_t.nodes[1].x
  min_y = M3_t.nodes[1].y
  max_x = M3_t.nodes[1].x
  max_y = M3_t.nodes[1].y

  for i in M3_t.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

  #Adding support
  for i in M3_t.nodes:
    if(i.z == 0):
      if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
        i.s_x = 1
        i.s_y = 1
        i.s_z = 1
        M3_t.support_x.append(i.n)
        M3_t.support_y.append(i.n)
        M3_t.support_z.append(i.n)


  mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny


  #Optimization preparation:
  x0 = []
  y0 = []
  z0 = []
  for i in M3_t.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
  coord0 = x0 + y0 + z0
  nNodes = int(len(coord0)/3)

  #bounds:
  lb = -np.inf*np.ones(len(coord0))
  ub = np.inf*np.ones(len(coord0))

  for i in M3_t.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

  for i in M3_t.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
  for i in M3_t.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


  bounds = Bounds(lb, ub)

  #Defining 2kN forces
  f = Point_load_at_centre(2, M3_t)

  result = minimize(Energy, coord0, args = (f, M3_t), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
  res = result.x
  res.tolist()
  data_M3_t.append(res)
  thickness_M3_t.append(thickness)
  energies_M3_t.append(result.fun)
  z_centre_M3_t.append(res[2*nNodes + mid_node - 1])
  results_M3_t.append(result)
  lat_M3_t.append(M3_t)

1
2
3
4
5
6
7
8
9
10


In [0]:
from numpy import save
save('data_M3_t.npy', data_M3_t)
save('thickness_M3_t.npy', thickness_M3_t)
save('energies_M3_t.npy', energies_M3_t)
save('z_centre_M3_t.npy', z_centre_M3_t)
save('results_M3_t.npy', results_M3_t)
save('lat_M3_t.npy', lat_M3_t)

In [0]:
print('hello world')

hello world
